In [5]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pandas import unique
import json
from pandas.io.json import json_normalize

In [14]:
indicators = pd.read_csv('indicators_by_company.csv')

In [197]:
indicators.head()

,company_id,indicator_id,2010,2011,2012,2013,2014,2015,2016
0,1000045,AccountsPayableAndAccruedLiabilitiesCurrentAnd...,NaN,6612429.0,7405579.0,8924919.0,7841070.0,5839000.0,NaN
1,1000045,AccumulatedDepreciationDepletionAndAmortizatio...,NaN,NaN,2111343.0,2242703.0,2236449.0,2462000.0,NaN
2,1000045,AdjustmentForAmortization,NaN,NaN,-11482251.0,-13490892.0,-13852305.0,-13811000.0,NaN
3,1000045,Assets,NaN,257236034.0,263835468.0,283429579.0,302528591.0,325309000.0,NaN
4,1000045,AssetsHeldForSaleAtCarryingValue,NaN,1373001.0,1203664.0,1696330.0,NaN,NaN,NaN


In [24]:
comp_name = pd.read_json('companies.json')
comp_name.head()
comp_name.dtypes

company_id         int64
name_latest       object
names_previous    object
dtype: object

In [39]:
df = pd.merge(comp_name,indicators,on='company_id')
df.head()

,company_id,name_latest,names_previous,indicator_id,2010,2011,2012,2013,2014,2015,2016
0,1000045,Nicholas Financial Inc,None,AccountsPayableAndAccruedLiabilitiesCurrentAnd...,NaN,6612429.0,7405579.0,8924919.0,7841070.0,5839000.0,NaN
1,1000045,Nicholas Financial Inc,None,AccumulatedDepreciationDepletionAndAmortizatio...,NaN,NaN,2111343.0,2242703.0,2236449.0,2462000.0,NaN
2,1000045,Nicholas Financial Inc,None,AdjustmentForAmortization,NaN,NaN,-11482251.0,-13490892.0,-13852305.0,-13811000.0,NaN
3,1000045,Nicholas Financial Inc,None,Assets,NaN,257236034.0,263835468.0,283429579.0,302528591.0,325309000.0,NaN
4,1000045,Nicholas Financial Inc,None,AssetsHeldForSaleAtCarryingValue,NaN,1373001.0,1203664.0,1696330.0,NaN,NaN,NaN


In [235]:
df2016 = df.ix[:,[0,1,2,3,10]]
df2016 = df2016[df2016['2016'].notnull() == True]
df2016 = df2016[(df2016['indicator_id'] == 'SalesRevenueNet')|(df2016['indicator_id'] == 'CostOfRevenue')]
df2016gr = df2016gr.groupby('name_latest').count()
namelist =[]
for name in df2016gr.index:
    if df2016gr.loc[name,'2016'] > 1:
        namelist.append(name)
namelist

NameError: name 'df2016gr' is not defined

In [238]:
df2016.sort_values('2016',ascending=False)
newdf = []
for comp in namelist:
    sales = pd.DataFrame(df2016.loc[(df2016['name_latest']== comp) & (df2016['indicator_id'] == 'SalesRevenueNet'),'2016']).iloc[0,0]
    costs = pd.DataFrame(df2016.loc[(df2016['name_latest']== comp) & (df2016['indicator_id'] == 'CostOfRevenue'),'2016']).iloc[0,0]
    newdf.append({'comp_name':comp,
                 'profit': sales - costs
                  
                 })
pd.DataFrame(newdf)

,comp_name,profit
0,Avnet Inc,3.037511e+09
1,Coach Inc,3.051300e+09
2,G&K Services Inc,3.349740e+08
3,Linear Technology Corp /CA/,1.080135e+09


In [102]:
df2016[(df2016['name_latest'] == 'Avnet Inc') & (df2016['indicator_id'] == 'Assets')]

,company_id,name_latest,names_previous,indicator_id,2016
1726239,8858,Avnet Inc,None,Assets,1.123980e+10


In [157]:
hey = df2016.loc[(df2016['name_latest']== 'Avnet Inc')&((df2016['indicator_id'] == 'Assets')),'2016']
hey= pd.DataFrame(hey)
hey.iloc[0,0]

11239805000.0

In [184]:
pd.DataFrame(df2016.loc[(df2016['name_latest']== comp) & (df2016['indicator_id'] == 'SalesRevenueNet'),'2016']).iloc[0,0]

26219279000.0

In [185]:
df2016['name_latest'].unique()

array(['Oclaro, Inc.', 'Coach Inc', 'Seagate Technology PLC',
       'Cimpress N.V.', 'G&K Services Inc', 'Linear Technology Corp /CA/',
       'Avnet Inc'], dtype=object)

In [233]:
df2016.loc['Avnet Inc','2016']

2